In [1]:
import tensorflow as tf
import random 
import numpy as np

In [1]:
# let us take simple example of sequence. 
# Assume we have 5 sequencies, each of length 3. Depended sequence
# will compute average over all prior sequences, 
# round it to whole integer and will take last digit. 
# We do it so this is classifiction task as is illustrated in many places. 
# Here is example of a sequence:

# x_1 = (1,2,3), y_1 = (1+2+3)/3 = 2
# x_2 = (5,6,7), y_2 = 4
# x_3 = (8,9,10), y_3 = 6
# x_4 = (322,425,727), y_4 = 7
# x_5 = (12, 12, 43), y_5 = 6


In [2]:
#constants
VECTOR_SIZE = 3
SEQUENCE_LENGTH = 5
BATCH_SIZE = 7
STATE_SIZE = 40
NUM_CLASSES = 10
LEARNING_RATE = 0.01

In [3]:
def get_data(n = 210):
    lower_bound = 0
    upper_bound = 3000
    x = [random.choice(range(lower_bound, upper_bound)) for _ in range(n)]
    return x

def y_from_x(x):
    y = [round(np.mean(x[0:el,:].flatten()))%10 for el in range(1,len(x)+1)]
    return y

In [ ]:
# we need to generate whole epoch  i.e.  collection of batches.
def get_epoch(n_batches = 100000):
    raw_x = get_data(n = n_batches*SEQUENCE_LENGTH*VECTOR_SIZE*BATCH_SIZE)
    reshaped_x =np.reshape(raw_x, (BATCH_SIZE, -1, VECTOR_SIZE))
    split_x = np.split(reshaped_x, reshaped_x.shape[1]/SEQUENCE_LENGTH, axis = 1)
    split_y = list()
    for el in split_x:
        split_y.append([y_from_x(el[sl,:,:]) for sl in range(len(el)) ])
    split_y =[np.array(el) for el in split_y]
    
    for sample_x, sample_y in zip(split_x, split_y):
        yield sample_x, sample_y
    

In [5]:
# NN setup
# we need placeholders for the input
x = tf.placeholder(tf.float32, [BATCH_SIZE, SEQUENCE_LENGTH, VECTOR_SIZE],
                   name='input_placeholder')
y = tf.placeholder(tf.int32, [BATCH_SIZE, SEQUENCE_LENGTH,],
                   name='labels_placeholder')
init_state = tf.zeros([BATCH_SIZE, STATE_SIZE])

rnn_inputs = tf.unstack(x, axis = 1)
y_as_list = tf.unstack(y, axis=1)

cell = tf.contrib.rnn.BasicLSTMCell(STATE_SIZE, state_is_tuple = True)
rnn_outputs, final_state = tf.contrib.rnn.static_rnn(cell, rnn_inputs,
                            initial_state=(init_state,init_state))

# we need to process output further in order to get it in the form we need. 
# output of each cell has to be one hot vector of length 10. 
# 0 = [1,0,0,0,0,0,0,0,0,0], ...,
# 9 =[0,0,0,0,0,0,0,0,0,1]. We should use softmax and let us use
# cross entropy as loss function.

# Let us define output 
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [STATE_SIZE, NUM_CLASSES])
    b = tf.get_variable('b', [NUM_CLASSES], initializer=tf.constant_initializer(0.0))
logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]

predictions = [tf.nn.softmax(logit) for logit in logits]

# Now let us deal with losses
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit) for \
          logit, label in zip(logits, y_as_list)]
# this requction is along sequnce length
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(LEARNING_RATE).minimize(total_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [6]:
def training(n_epoch = 10):
    
    epoch_loss_list = list()
    for i, el in enumerate(range(n_epoch)):
        epoch = get_epoch(n_batches=1000000)
        epoch_loss = 0
        for X,Y in epoch:
            batch_total_loss = sess.run([total_loss, train_step],
                                        feed_dict = {x:X,y:Y})
            epoch_loss += np.sum(batch_total_loss[0])
        epoch_loss_list.append(epoch_loss)
    return epoch_loss_list
        

In [ ]:
training_run = training(n_epoch = 1)

In [46]:
def compute_accuracy(n_batches = 1000):
    epoch = get_epoch(n_batches= n_batches)
    total_correct = 0
    for X, Y in epoch:
        predictions_result = sess.run([predictions], feed_dict = {x:X})
        total_correct += count_correct(predictions_result[0], Y)
    return total_correct/(n_batches*BATCH_SIZE)

In [47]:
compute_accuracy()

0.00014285714285714287

In [39]:
# let us evaluate performance of our network. Let us simply count number 
# of correct sequences
def count_correct(est_Y, true_Y):
    # est_Y will get list of (batch_size x num_of_classes) array
    # length of est_Y is length of our sequence.
    # true_Y is of batch_size x sequence_length form.
    # this method will return number of coinciding sequences.
    est_Y = back_to_y(est_Y)
    counter = 0
    for el in range(BATCH_SIZE):
        if np.array_equal(est_Y[el,:], true_Y[el,:]):
            counter+=1
    return counter

In [11]:
# we need function to translate back to y 
def back_to_y(batch):
    # output will be of the shape (batch_size x sequence_length)
    # we are getting list with one element of list per our 
    # sequence element
    result = np.zeros(shape=(7,5))
    for i, el in enumerate(batch):
        result[:,i] = np.argmax(el, axis =1)
    return result